In [1]:
import cv2
import torch
import numpy as np

In [2]:
customyolov5s = torch.hub.load('','custom', path='weightHedect/nexgen_hedec_s1.pt', source='local')

YOLOv5  v6.1-222-gb4e823f Python-3.8.12 torch-1.10.0 CUDA:0 (NVIDIA GeForce GTX 1060 6GB, 6144MiB)

Fusing layers... 
Model summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [3]:
ori_img = cv2.imread('test_image/test_img_1.jpg')

cv2.imshow('This img',ori_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")
print(device)

Setup complete. Using torch 1.10.0 (NVIDIA GeForce GTX 1060 6GB)
cuda


In [5]:
model = customyolov5s.to(device)

In [6]:
img = ori_img
results = model(img)

In [7]:
# print(results.pandas().xyxy[0])

presults = results.pandas().xyxy[0]


results_cpu = results.xyxyn[0].cpu().numpy()

i = 0

# print(presults.iloc[:,:-1])

print(presults)
print(presults.loc[3])


# print(results_cpu)

         xmin        ymin        xmax        ymax  confidence  class     name
0  869.907776  135.856628  962.590515  258.334320    0.853695      1     head
1  872.061279  133.206192  962.023438  196.929916    0.819372      0  hardhat
2  705.453064  167.756943  790.738098  248.270935    0.429588      0  hardhat
3  707.362793  167.531113  787.806152  250.057678    0.316106      1     head
xmin          707.362793
ymin          167.531113
xmax          787.806152
ymax          250.057678
confidence      0.316106
class                  1
name                head
Name: 3, dtype: object


In [12]:
def bb_intersection_over_union(presults, nameA,nameB):
	boxA = [presults['xmin'][nameA],presults['ymin'][nameA],presults['xmax'][nameA],presults['ymax'][nameA]]
	boxB = [presults['xmin'][nameB],presults['ymin'][nameB],presults['xmax'][nameB],presults['ymax'][nameB]]

	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
	boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

In [14]:
def check_helmetWear(presults):
    head_list = []
    hardhat_list = []
    for i in range(len(presults)):
        if presults['class'][i] == 0:
            hardhat_list.append(i)
        elif presults['class'][i] == 1:
            head_list.append(i)

    for i in range(len(head_list)):
        for j in range(len(hardhat_list)):
            iou_result = bb_intersection_over_union(presults,head_list[i],hardhat_list[j])
            print(iou_result)
        
    
    
            

In [15]:
check_helmetWear(presults)

0.4781444114541731
0.0
0.0
0.9223915611187468


In [8]:
# cv2.rectangle(img, (115,228),(281,415),(0, 255, 0),2)



def printTheBoxYEAAAH(img,predresultinpandas):
    printed_img = img
    for i in range(len(predresultinpandas)):
        printed_img = cv2.rectangle(printed_img, (int(predresultinpandas['xmin'][i]),int(predresultinpandas['ymin'][i])),(int(predresultinpandas['xmax'][i]),int(predresultinpandas['ymax'][i])),(0, 255, 0),2)
        printed_img = cv2.putText(printed_img,predresultinpandas['name'][i],(int(predresultinpandas['xmin'][i]),int(predresultinpandas['ymin'][i])),cv2.FONT_HERSHEY_SIMPLEX,0.9,(0, 255, 0),2)

    return printed_img
    


printedimg = printTheBoxYEAAAH(img,presults)

cv2.imshow('This img',img)
cv2.waitKey(0)
cv2.destroyAllWindows()